In [1]:
"""
READ BEFORE START:

    DATA format:
    - In each animal's file, create a subfoler for each day (called d1, d2 etc)
    - In each day's subfolder, have a folder called "files" containing all the recorded time points, voltage recording and behavior
    - suite2p folder is created automatically with suite2p, needs to have mat file for all information
    - can have the reference folder in this subfolder as well
    - running this file will make a result folder in this folder

"""

import argparse
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io as sio
import seaborn as sns
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.metrics import (
    accuracy_score,
    silhouette_score,
    adjusted_rand_score,
    silhouette_samples,
)
from sklearn.cluster import AgglomerativeClustering, SpectralClustering, KMeans
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import (
    ModelDesc,
    EvalEnvironment,
    Term,
    EvalFactor,
    LookupFactor,
    dmatrices,
    INTERCEPT,
)
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.colorbar as colorbar
import sys

from s2p_utils.data_loader import DataLoader
from s2p_utils.processing_utils import (
    correct_overlapping_cells_across_planes,
    correct_timestamps,
    get_cell_only_activity,
    extract_events,
    get_corrected_F,
    extract_interest_time_intervals,
    extract_imaging_ts_around_events,
    normalize_signal,
    extract_Fave_around_events,
    reorder_clusters,
)
from plot_utils import (
    plot_raw_licks,
    plot_average_PSTH_around_interest_window,
    plot_individual_cells_activity,
    plot_PC_screenplot,
    plot_PCs,
    make_silhouette_plot,
    plot_activity_clusters,
    plot_cluster_pairs,
)


logger = logging.getLogger(__name__)


def parse_args():
    """Parses arguments from command line."""
    parser = argparse.ArgumentParser(description="Suite2p result analyzer.")

    parser.add_argument(
        "--data_dir",
        type=str,
        required=False,
        # default="/Users/mzhou/Library/CloudStorage/OneDrive-UCSF/MZ_hpc_prism_M4/d5/",
        default="Z:\\2p\\experiment1\\MZ_hpc_prism_M6\\d7\\",
        help="Directory containing all the required files.",
    )
    parser.add_argument(
        "--result_folder",
        type=str,
        default="result",
        help="folder for saving figures and results",
    )
    parser.add_argument(
        "--num_planes",
        type=int,
        default="4",
        help="Number of planes recorded during each session",
    )
    parser.add_argument(
        "--min_cell_prob",
        type=float,
        default=0.5,
        help="Minimum probability to identify an ROI as a cell.",
    )
    parser.add_argument(
        "--pre_cue_window",
        type=float,
        default=3,
        help="Interested time region before a cue starts. (Second)",
    )
    parser.add_argument(
        "--post_cue_window",
        type=float,
        default=17,
        help="Interested time region after a cue starts. (Second)",
    )
    parser.add_argument(
        "--delay_to_reward",
        type=int,
        default=3,
        help="Number of seconds from cue onset to reward onset",
    )
    parser.add_argument(
        "--framerate", type=int, default=5, help="Average frame rate, 5hz"
    )
    parser.add_argument(
        "--trial_types",
        type=list,
        default=["CS1+", "CS2+", "CS3-"],
        help="Trial types of the experiment",
    )
    parser.add_argument(
        "--neucoeff",
        type=float,
        default=0.7,
        help="neuropil coefficient factor",
    )
    parser.add_argument(
        "--cell_threshold",
        type=int,
        default=10,
        help="threshold percentage difference between F and Fneu to count as a valid cell",
    )
    return parser.parse_args()


def associate_cells_with_intervals(
    interval: list,
    time: float,
    cell_indices_with_activities: "dict[int, set]",
    intervals_with_detected_cells: "dict[int, set]",
    cell_idx: int,
):
    for interval_idx, (start, end) in enumerate(interval):
        if time >= start and time <= end:
            if cell_idx not in cell_indices_with_activities:
                cell_indices_with_activities[cell_idx] = set()
            cell_indices_with_activities[cell_idx].add(interval_idx)

            if interval_idx not in intervals_with_detected_cells:
                intervals_with_detected_cells[interval_idx] = []
            intervals_with_detected_cells[interval_idx].add(cell_idx)

c:\Users\mzhou9\Anaconda3\envs\py3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\mzhou9\Anaconda3\envs\py3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [10]:
# Load data
args = parse_args()
print(args)
data_loader = DataLoader(args.data_dir, args.num_planes)
args.data_dir = "Z:\\2p\\experiment1\\MZ_hpc_prism_M4\\d5"

# Make a result folder if if didn't exist
result_dir = os.path.join(args.data_dir, args.result_folder)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
file_dir = os.path.join(args.data_dir, "files")

usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR]
                             [--result_folder RESULT_FOLDER]
                             [--num_planes NUM_PLANES]
                             [--min_cell_prob MIN_CELL_PROB]
                             [--pre_cue_window PRE_CUE_WINDOW]
                             [--post_cue_window POST_CUE_WINDOW]
                             [--delay_to_reward DELAY_TO_REWARD]
                             [--framerate FRAMERATE]
                             [--trial_types TRIAL_TYPES] [--neucoeff NEUCOEFF]
                             [--cell_threshold CELL_THRESHOLD]
ipykernel_launcher.py: error: argument --framerate: invalid int value: 'c:\\Users\\mzhou9\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-32448nypNBgj47ZiQ.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\mzhou9\Anaconda3\envs\py3\lib\argparse.py", line 2422, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: 'c:\\Users\\mzhou9\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-32448nypNBgj47ZiQ.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mzhou9\Anaconda3\envs\py3\lib\argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "c:\Users\mzhou9\Anaconda3\envs\py3\lib\argparse.py", line 2006, in _parse_known_args
    start_index = consume_optional(start_index)
  File "c:\Users\mzhou9\Anaconda3\envs\py3\lib\argparse.py", line 1946, in consume_optional
    take_action(action, args, option_string)
  File "c:\Users\mzhou9\Anaconda3\envs\py3\lib\argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "c:\Us

TypeError: object of type 'NoneType' has no len()

In [3]:
args

NameError: name 'args' is not defined